In [1]:
import sympy as sp
from sympy.abc import x, y, z, w
from nbsupport import md

In [2]:
def _subs(EXPR, repl):
    return EXPR.subs(repl, simultaneous=True)

VEC2 = sp.Matrix([x, y])
VEC3 = sp.Matrix([x, y, z])
VEC4 = sp.Matrix([x, y, z, w])

def Vec2(X, Y):       return _subs(VEC2, {x: X, y: Y})
def Vec3(X, Y, Z):    return _subs(VEC3, {x: X, y: Y, z: Z})
def Vec4(X, Y, Z, W): return _subs(VEC4, {x: X, y: Y, z: Z, w: W})

# Todo: extend the definition of Homogeneous to Matrices

def Homogeneous(p, w=1):
    return (p * w).col_join(sp.Matrix([w]))

def Cartesian(h):
    return h[0:3, -1] / h[3,0]

In [3]:
def solve_mapping(X, Y):
    n = X.shape[0]
    T = sp.Matrix(sp.MatrixSymbol('T', n, n))
    elements = T.values()
    return T.subs(sp.solve(T * X - Y, elements))

bases = [[1,0,0],[0,1,0],[0,0,1],[0,0,0]]

def solve_mapping2(*Y):
    Y = Homogeneous(Vec3(*Y), w)
    Y = sp.Matrix([Y.T.subs(list(zip([x, y, z], base))) for base in bases]).T
    return solve_mapping(X, Y)

In [4]:
scale_symbols = sx, sy, sz = sp.var('s_x s_y s_z')
SCALE = solve_mapping2(sx*x, sy*y, sz*z)

translate_symbols = tx, ty, tz = sp.var('t_x t_y t_z')
TRANSLATE = solve_mapping2(x + tx, y + ty, z + tz)

theta = sp.var(r'\theta')
c, s = sp.cos(theta), sp.sin(theta)
R = sp.Matrix([[c, s], [-s, c]])

RX = R * sp.Matrix([y, z])
ROTATE_X = solve_mapping2(x, RX[0], RX[1])

RY = R * sp.Matrix([z, x])
ROTATE_Y = solve_mapping2(RY[1], y, RY[0])

RZ = R * sp.Matrix([x, y])
ROTATE_Z = solve_mapping2(RZ[0], RZ[1], z)

md(r'$$\begin {align*}',
   'SCALE &=', SCALE, r'\\',
   'TRANSLATE &=', TRANSLATE, r'\\',
   'Rotate_x &=', ROTATE_X, r'\\',
   'Rotate_y &=', ROTATE_Y, r'\\',
   'Rotate_z &=', ROTATE_Z,
   r'\end {align*}$$')

NameError: name 'X' is not defined

In [ ]:
def M32(ARGS, EXPR):
    f = sp.lambdify(ARGS, EXPR, 'numpy')
    return lambda *args: np.array(f(*args), dtype=np.float32)

def V32(ARGS, EXPR):
    f = M32(ARGS, EXPR)
    return lambda *args: f(*args).reshape((len(ARGS),))

In [ ]:
scale = M32(scale_symbols, SCALE)
translate = M32(translate_symbols, TRANSLATE)
rotate_x = M32([theta], ROTATE_X)
rotate_y = M32([theta], ROTATE_Y)
rotate_z = M32([theta], ROTATE_Z)

In [ ]:
LookAtVectors = [Eye, Side, Head, Forward] = [
    sp.var('I_x I_y I_z'), sp.var('S_x S_y S_z'),
    sp.var('H_x H_y H_z'), sp.var('F_x F_y F_z')]
[I, S, H, F] = [sp.Matrix(3, 1, M) for M in LookAtVectors]

LOOK_AT_TRANSLATE = TRANSLATE.subs({ tx: -I_x, ty: -I_y, tz: -I_z })
# md('$$', LOOK_AT_TRANSLATE, '$$')
# print(sp.ask(sp.Q.orthogonal(LOOK_AT_TRANSLATE)))
INV_LOOK_AT_TRANSLATE = sp.Matrix(sp.BlockMatrix([[S, H, -F, sp.zeros(3, 1)]])).col_join(sp.Matrix([[0, 0, 0, 1]]))
LOOK_AT_ROTATE = INV_LOOK_AT_TRANSLATE.T  # 回転行列は正規直交行列なので，逆行列は転置行列

LOOK_AT = LOOK_AT_ROTATE * LOOK_AT_TRANSLATE
md('$$LookAt = ', LOOK_AT, '$$')
lookat = M32(Eye + Forward + Side + Head, LOOK_AT)

In [ ]:
M = sp.MatrixSymbol('M', 4, 4)
sp.Matrix(M).values()